In [1]:
import pandas as pd
import numpy as np
import os

### Review contents of all `./HFMG/Compressed`
* 29 Files or parent directories.
* We are interested only in data having a timestamp. 

In [2]:
struct = {}
path = './HFHS Data/Compressed/'
for root, dirs, files in os.walk(path):
    if root != path: 
        struct[root] = files

struct = dict(sorted(struct.items()))

In [3]:
assert len(struct) == 29
for k, v in struct.items():
    print(str(len(v)) + '\t' + k)

1	./HFHS Data/Compressed/Complete ED Provider Notes for 2014 Patients
4	./HFHS Data/Compressed/Complete ED Provider Notes for 2015-2016 Patients
1	./HFHS Data/Compressed/WSU_HASHED_ALL_CHIEF_COMPLAINTS
1	./HFHS Data/Compressed/WSU_HASHED_ALL_CHIEF_COMPLAINTS_NEW
1	./HFHS Data/Compressed/WSU_HASHED_ALL_CHIEF_COMPLAINTS_inc_unspecified
1	./HFHS Data/Compressed/WSU_HASHED_ALL_INPATIENT_ADMIT_DISCHARGE_TIMES
1	./HFHS Data/Compressed/WSU_HASHED_ATMO_METRICS_3
1	./HFHS Data/Compressed/WSU_HASHED_ATMO_METRICS_4
1	./HFHS Data/Compressed/WSU_HASHED_BASE_CSN_SELECT
1	./HFHS Data/Compressed/WSU_HASHED_BASE_CSN_SELECT_FirstNonEDDepartment
1	./HFHS Data/Compressed/WSU_HASHED_BASE_MRN_SELECT
1	./HFHS Data/Compressed/WSU_HASHED_BED_PATIENT_MOVEMENT
1	./HFHS Data/Compressed/WSU_HASHED_BED_REQUEST_INFO
6	./HFHS Data/Compressed/WSU_HASHED_CSN_MRN_PATIENT_MOVEMENT
1	./HFHS Data/Compressed/WSU_HASHED_DX_LIST
1	./HFHS Data/Compressed/WSU_HASHED_EVENTS
1	./HFHS Data/Compressed/WSU_HASHED_LAB_TEST_INFO
1	./H

In [4]:
targets = []
for root, files in struct.items():
    for file in sorted(files): 
        if 'note' in root.lower() or 'note' in file.lower():
            print(os.path.join(root, file))
        else: 
            targets.append(os.path.join(root, file))

./HFHS Data/Compressed/Complete ED Provider Notes for 2014 Patients/2014 ED Provider Notes.txt
./HFHS Data/Compressed/Complete ED Provider Notes for 2015-2016 Patients/Provider Notes 2015-01-01 through 2015-06-30.txt
./HFHS Data/Compressed/Complete ED Provider Notes for 2015-2016 Patients/Provider Notes 2015-07-01 through 2015-12-31.txt
./HFHS Data/Compressed/Complete ED Provider Notes for 2015-2016 Patients/Provider Notes 2016-01-01 through 2016-06-30.txt
./HFHS Data/Compressed/Complete ED Provider Notes for 2015-2016 Patients/Provider Notes 2016-07-01 through 2016-12-31.txt
./HFHS Data/Compressed/WSU_HASHED_NOTE_ED_Notes_only/WSU_HASHED_NOTE_ED_Notes_only.txt
./HFHS Data/Compressed/WSU_HASHED_NOTE_ED_Provider_Notes_only/WSU_HASHED_NOTE_ED_Provider_Notes_only.txt
./HFHS Data/Compressed/WSU_HASHED_NOTE_ED_other_Notes/WSU_HASHED_NOTE_ED_other_Notes.txt
./HFHS Data/Compressed/WSU_HASHED_NOTE_INFO/WSU_HASHED_NOTE_INFO.csv
./HFHS Data/Compressed/WSU_HASHED_NOTE_INFO_NEW/WSU_HASHED_NOTE_INF

In [5]:
len(targets)

33

In [6]:
targets

['./HFHS Data/Compressed/WSU_HASHED_ALL_CHIEF_COMPLAINTS/WSU_HASHED_ALL_CHIEF_COMPLAINTS.csv',
 './HFHS Data/Compressed/WSU_HASHED_ALL_CHIEF_COMPLAINTS_NEW/WSU_HASHED_ALL_CHIEF_COMPLAINTS_NEW.csv',
 './HFHS Data/Compressed/WSU_HASHED_ALL_CHIEF_COMPLAINTS_inc_unspecified/WSU_HASHED_ALL_CHIEF_COMPLAINTS_inc_unspecified.csv',
 './HFHS Data/Compressed/WSU_HASHED_ALL_INPATIENT_ADMIT_DISCHARGE_TIMES/WSU_HASHED_ALL_INPATIENT_ADMIT_DISCHARGE_TIMES.csv',
 './HFHS Data/Compressed/WSU_HASHED_ATMO_METRICS_3/WSU_HASHED_ATMO_METRICS_3.csv',
 './HFHS Data/Compressed/WSU_HASHED_ATMO_METRICS_4/WSU_HASHED_ATMO_METRICS_4.csv',
 './HFHS Data/Compressed/WSU_HASHED_BASE_CSN_SELECT/WSU_HASHED_BASE_CSN_SELECT.csv',
 './HFHS Data/Compressed/WSU_HASHED_BASE_CSN_SELECT_FirstNonEDDepartment/WSU_HASHED_BASE_CSN_SELECT_FirstNonEDDepartment.csv',
 './HFHS Data/Compressed/WSU_HASHED_BASE_MRN_SELECT/WSU_HASHED_BASE_MRN_SELECT.csv',
 './HFHS Data/Compressed/WSU_HASHED_BED_PATIENT_MOVEMENT/WSU_HASHED_BED_PATIENT_MOVEMEN

Pull out the first three rows of each file and inspect for: 
* presence of headers,
* presence/position of `CSN_HASH` (visit ID),
* presence/position of timestamp.

In [ ]:
for i, file in enumerate(targets): 
    print(f'File {i + 1}: {file}\n')
    with open(file) as f:
        [print('\t' + f.readline().strip()) for _ in range(3)]
    print('\n')

### I. `WSU_HASHED_LAB_TEST_INFO.csv`

In [8]:
file = "./HFHS Data/Compressed/WSU_HASHED_LAB_TEST_INFO/WSU_HASHED_LAB_TEST_INFO.csv"
with open(file, encoding="utf-8-sig", newline="") as f:
    lines = [line.rstrip("\r\n") for line in f]

assert len(lines) == 14_679_799

In [9]:
import polars as pl

df = pl.DataFrame({"row": lines})

##### 1. Extract `CSN_HASH` and `CPT_CODE`

In [10]:
df = df.with_columns(
    pl.col("row")
    .str.split_exact(",", 1)
    .struct.rename_fields(["CSN_HASH", "other"])
    .alias("fields")
).unnest("fields").drop("other")

df = df.with_columns(
    pl.col("row")
    .str.reverse()
    .str.split_exact(",", 1)
    .struct.rename_fields(["CPT_CODE", "other"])
    .alias("fields")
).unnest("fields").drop("other")

df = df.with_columns(
    pl.col("CPT_CODE")
    .str.reverse()
)

assert df["CSN_HASH"].str.len_chars().mean() == 66
assert df["CSN_HASH"].null_count() == 0
assert df.filter(pl.col("CPT_CODE") == "NULL").shape[0] == 0
assert df["CPT_CODE"].null_count() == 0

df.head()

row,CSN_HASH,CPT_CODE
str,str,str
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB747"""
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB747"""
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB276"""
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB462"""
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB462"""


##### 2. Count `datetime` in each row
Then subset the results.

In [ ]:
#datetime_regex = r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}$"
datetime_regex = r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}"

df = df.with_columns(
    pl.col("row").str.count_matches(datetime_regex).alias("dt_count")
)

df['dt_count'].value_counts().sort("count", descending=True)

In [12]:
df['dt_count'].value_counts().sort("count", descending=True)

dt_count,count
u32,u32
4,8988198
2,5098101
3,589766
1,3734


In [118]:
#group_4 = df.filter(pl.col("dt_count") == 4).drop("dt_count")
#group_2 = df.filter(pl.col("dt_count") == 2).drop("dt_count")
#group_3 = df.filter(pl.col("dt_count") == 3).drop("dt_count")
group_1 = df.filter(pl.col("dt_count") == 1).drop("dt_count")
#assert group_4.shape[0] + group_2.shape[0] + group_3.shape[0] + group_1.shape[0] == df.shape[0]

##### 3. Work date cols on `group_4`
All records have four `datetime` substrings which is the maximum number expected.
```    
RESULT_TIME
TEST_REQUEST_TIME
TEST_START_TIME
TEST_COMPLETE_TIME
```    

In [14]:
group_4 = group_4.with_columns(
    pl.col("row")
    .str.slice(
        pl.col("row")
        .str.find(datetime_regex)
    )
    .str.splitn(",", 5)
    .struct.rename_fields(["RESULT_TIME", "RESULT_TYPE", "RESULT_FLAG", "ORD_VALUE", "OTHER"])
    .alias("t_break")
).unnest("t_break").drop(["RESULT_TIME", "RESULT_TYPE", "RESULT_FLAG", "OTHER"])

group_4.head()

row,CSN_HASH,CPT_CODE,ORD_VALUE
str,str,str,str
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB747""","""0.11"""
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB747""","""7.73"""
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB276""","""AB POSITIVE"""
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB462""","""Blood"""
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB462""","""Blood"""


**Note**: We should consider returning statistics on the position of the first datetime in order to estimate whether or not a `RESULT_TIME` is present. 

Assumption: Just because there are three other times, doesn't mean that the test yielded a valid result; this can be evaluated by attempting to extract values at the `ORD_VALUE` and testing for nulls.

In [15]:
group_4.with_columns(
    pl.col("row")
    .str.find(datetime_regex)
    .alias("first_dt_pos")
)["first_dt_pos"].describe()

statistic,value
str,f64
"""count""",8.988198e6
"""null_count""",0.0
"""mean""",127.70241
"""std""",28.825124
"""min""",103.0
"""25%""",114.0
"""50%""",123.0
"""75%""",129.0
"""max""",385.0


In [16]:
group_4.shape[0] / df.shape[0]

0.6122834515649703

In [17]:
fields = ["RESULT_TIME", "TEST_REQUEST_TIME", "TEST_START_TIME", "TEST_COMPLETE_TIME"]

group_4 = group_4.with_columns(
    pl.col("row")
    .str.extract_all(datetime_regex)
    .list.to_struct(fields=fields)
    .alias("dts")
).unnest("dts")

group_4 = group_4.with_columns(
    pl.col("^.*_TIME$")
    .str.to_datetime()
)

group_4.head()

row,CSN_HASH,CPT_CODE,ORD_VALUE,RESULT_TIME,TEST_REQUEST_TIME,TEST_START_TIME,TEST_COMPLETE_TIME
str,str,str,str,datetime[μs],datetime[μs],datetime[μs],datetime[μs]
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB747""","""0.11""",2014-08-15 18:49:00,2014-08-15 17:10:00,2014-08-15 17:09:00,2014-08-15 18:49:00
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB747""","""7.73""",2014-08-15 21:59:00,2014-08-15 17:10:00,2014-08-15 20:09:00,2014-08-15 21:59:00
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB276""","""AB POSITIVE""",2014-08-15 18:11:00,2014-08-15 17:10:00,2014-08-15 17:10:00,2014-08-15 18:11:00
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB462""","""Blood""",2014-08-20 15:53:00,2014-08-15 17:12:00,2014-08-15 17:12:00,2014-08-20 15:53:00
"""0x6866B1E321722786E0C3CFBAF02E…","""0x6866B1E321722786E0C3CFBAF02E…","""LAB462""","""Blood""",2014-08-20 10:36:00,2014-08-15 17:12:00,2014-08-15 17:27:00,2014-08-20 10:36:00


In [18]:
group_4[fields].describe()

statistic,RESULT_TIME,TEST_REQUEST_TIME,TEST_START_TIME,TEST_COMPLETE_TIME
str,str,str,str,str
"""count""","""8988198""","""8988198""","""8988198""","""8988198"""
"""null_count""","""0""","""0""","""0""","""0"""
"""mean""","""2015-08-29 00:26:49.559569""","""2015-08-28 16:22:33.906623""","""2015-08-28 17:16:22.341771""","""2015-08-29 00:26:49.559569"""
"""std""",null,null,null,null
"""min""","""2013-12-03 00:00:00""","""2012-05-04 17:45:00""","""2012-05-04 17:45:00""","""2013-12-03 00:00:00"""
"""25%""","""2015-01-05 13:28:00""","""2015-01-05 12:08:00""","""2015-01-05 12:17:00""","""2015-01-05 13:28:00"""
"""50%""","""2015-08-28 21:15:00""","""2015-08-28 13:51:00""","""2015-08-28 13:59:00""","""2015-08-28 21:15:00"""
"""75%""","""2016-04-19 14:47:00""","""2016-04-19 06:48:00""","""2016-04-19 07:03:00""","""2016-04-19 14:47:00"""
"""max""","""2016-12-21 00:03:00""","""2016-12-20 23:57:00""","""2016-12-21 00:01:00""","""2016-12-21 00:03:00"""


In [19]:
group_4.with_columns(
    pl.when(
        pl.col.TEST_REQUEST_TIME <= pl.col.TEST_START_TIME, 
        pl.col.TEST_START_TIME <= pl.col.TEST_COMPLETE_TIME, 
        pl.col.TEST_COMPLETE_TIME <= pl.col.RESULT_TIME, 
    )
    .then(1)
    .otherwise(0)
    .alias("t_val_test")
)["t_val_test"].mean()

0.861753601778688

86% of records abide by this logic; therefore, we shall assign the min value (in this case, typically `TEST_REQUEST_TIME`) to the "test ordered" event and the max value (in this case, typically the `RESULT_TIME`) to the `ORD_VALUE` event.

In [ ]:
group_4 = group_4.select(
    pl.col("CSN_HASH"), 
    pl.min_horizontal(fields).alias("datetime_val"),
    pl.col("CPT_CODE").alias("cat"),
    pl.lit("test ordered").alias("val")
).vstack(
    group_4.select(
        pl.col("CSN_HASH"), 
        pl.max_horizontal(fields).alias("datetime_val"),
        pl.col("CPT_CODE").alias("cat"),
        pl.col("ORD_VALUE").alias("val")
    )
).unique()

In [133]:
group_4.head()

CSN_HASH,datetime_val,cat,val
str,datetime[μs],str,str
"""0x3502D3DDCFED393A57065B5DB87B…",2014-07-05 21:09:00,"""LAB293""","""test ordered"""
"""0xCA6333496B2A04C3958BBDA1C663…",2014-06-29 10:11:00,"""LAB15""","""test ordered"""
"""0xFC9BEED592C923C32ED7308CE278…",2015-05-21 18:37:00,"""LAB747""","""test ordered"""
"""0x9AC236D9873985DA48BD173BAA1F…",2015-11-25 00:00:00,"""LAB15""","""test ordered"""
"""0xFC209EF7EC8D39623CB879BB616F…",2016-09-10 01:09:00,"""LAB15""","""test ordered"""


In [134]:
group_4.tail()

CSN_HASH,datetime_val,cat,val
str,datetime[μs],str,str
"""0x5A8276FB63D77813AE4FB025C547…",2014-07-06 14:44:00,"""LAB144""","""test ordered"""
"""0x2C3E077F355F3A99F58BD94A3D3D…",2015-04-10 16:00:00,"""LAB15""","""test ordered"""
"""0x2C3CA8E8E3CA151990B3CD43BD99…",2016-04-23 15:12:00,"""LAB56""","""test ordered"""
"""0x1242DDF9A9DC1C1978B6E99F7EF8…",2016-09-20 16:28:00,"""LAB347""","""test ordered"""
"""0x4CF38D8B907C96C44AFB3234483F…",2015-06-04 07:30:00,"""LAB47""","""61"""


##### 4. Work date cols on `group_2`
Are the date columns next to each other? This implies the pattern `r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}\,\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}"`
* If columns are next to each other then `RESULT_TIME` is **not** one of the columns. 
* But does this imply that there is null/missing value in `ORD_VALUE`? Assumption is "yes"...

In [33]:
datetime_regex = r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}\,\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}"

group_2 = group_2.with_columns(
    pl.col("row").str.count_matches(datetime_regex).alias("dt_count")
)

group_2['dt_count'].value_counts().sort("count", descending=True)

dt_count,count
u32,u32
1,5098101


This result implies that there is null-missing value in `ORD_VALUE` position. There should be some verification that this assumption is true.

In [ ]:
group_2["row"].slice(0, 1).to_numpy()

In [ ]:
group_2["row"].slice(-1, 1).to_numpy()

We could consider counting number of `NULL` in each string in `group_2` before making a final decision that *all `group_2` consist only of 'test ordered' events and no 'test result' events*

Subsequent steps would then include extracting the datetime values, finding their minimum, and thenn formatting the results as was done for `group_4` subset.

In [46]:
group_2 = group_2.with_columns(
    pl.col("row").str.count_matches("NULL", literal=True).alias("NULL_count")
)

group_2['NULL_count'].value_counts().sort("count", descending=True)

NULL_count,count
u32,u32
13,3915946
11,1170663
14,11492


In [ ]:
group_2.filter(pl.col("NULL_count") == 14)["row"].slice(0, 1).to_numpy()

In [ ]:
group_2.filter(pl.col("NULL_count") == 11)["row"].slice(0, 1).to_numpy()

In [ ]:
group_2.filter(pl.col("NULL_count") == 13)["row"].slice(0, 1).to_numpy()

Let's verify that a series of ten consecutive `NULL` exist in each string:

In [63]:
group_2 = group_2.with_columns(
    pl.col("row").str.count_matches(10 * ',NULL' + ',', literal=True).alias("10_NULL_count")
)

group_2['10_NULL_count'].value_counts().sort("count", descending=True)

10_NULL_count,count
u32,u32
1,5098101


Since there are a series of ten consecutive `NULL` values we consider there to be no results in subset `group_2` and therefore process it *only* as "test requested" events.

In [84]:
fields = ["TEST_REQUEST_TIME", "TEST_START_TIME"]

group_2 = group_2.with_columns(
    pl.col("row")
    .str.extract_all(datetime_regex)
    .list.get(0)
    .str.split(by=",")
    .alias("dts")        
    .list.to_struct(fields=fields)
).unnest("dts")

group_2 = group_2.with_columns(
    pl.col("^.*_TIME$")
    .str.to_datetime()
)

group_2 = group_2.select(
    pl.col("CSN_HASH"), 
    pl.min_horizontal(fields).alias("datetime_val"),
    pl.col("CPT_CODE").alias("cat"),
    pl.lit("test ordered").alias("val")
).unique()

group_2.head()

CSN_HASH,datetime_val,cat,val
str,datetime[μs],str,str
"""0x86DA87B0958100F93A3006F37FBD…",2016-04-18 17:26:00,"""NUR478""","""test ordered"""
"""0xF170C1EB0DADE73ADFA3EC8BFA9F…",2016-10-31 11:17:00,"""VAS30414""","""test ordered"""
"""0xDE49729D1F01DCC9532F87B14301…",2015-06-07 20:54:00,"""PRO179B""","""test ordered"""
"""0x69BF04A04D5C1A0C3FA581475C0A…",2016-02-01 02:43:00,"""COR25""","""test ordered"""
"""0x79564F04EE725A209C4FC16C2367…",2014-07-18 00:20:00,"""EQ97""","""test ordered"""


##### 5. Work date cols on `group_3`
Are the date columns next to each other? This implies the pattern `r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}\,\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}\,\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}"`
* If columns are next to each other then `RESULT_TIME` is **not** one of the columns. 
* But does this imply that there is null/missing value in `ORD_VALUE`? Assumption is "yes"...

In [85]:
datetime_regex = r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}\,\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}\,\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}"

group_3 = group_3.with_columns(
    pl.col("row").str.count_matches(datetime_regex).alias("dt_count")
)

group_3['dt_count'].value_counts().sort("count", descending=True)

dt_count,count
u32,u32
1,589729
0,37


For results where `dt_count == 1` we can assume all of these records should be "test ordered" or verify using series of 10 NULL method. 

For results where `dt_count == 0` this implies that since there are only three total datetimes in the record and they are not one after another, this means that one of the datetimes is in the `RESULT_TIME` position. We should assume that these records have valid `ORD_VALUE` which needs to be extracted.

In [ ]:
group_3_1 = group_3.filter(pl.col("dt_count") == 1)

group_3_1 = group_3_1.with_columns(
    pl.col("row").str.count_matches(10 * ',NULL' + ',', literal=True).alias("10_NULL_count")
)

group_3_1['10_NULL_count'].value_counts().sort("count", descending=True)

10_NULL_count,count
u32,u32
1,589729


Since there are a series of ten consecutive `NULL` values we consider there to be no results in subset `group_3_1` and therefore process it *only* as "test requested" events.

In [108]:
datetime_regex = r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}"

fields = ["TEST_REQUEST_TIME", "TEST_START_TIME", "TEST_COMPLETE_TIME"]

group_3_1 = group_3_1.with_columns(
    pl.col("row")
    .str.extract_all(datetime_regex)
    .list.to_struct(fields=fields)
    .alias("dts")            
).unnest("dts")

group_3_1 = group_3_1.with_columns(
    pl.col("^.*_TIME$")
    .str.to_datetime()
)

group_3_1 = group_3_1.select(
    pl.col("CSN_HASH"), 
    pl.min_horizontal(fields).alias("datetime_val"),
    pl.col("CPT_CODE").alias("cat"),
    pl.lit("test ordered").alias("val")
).unique()

group_3_1.head()

CSN_HASH,datetime_val,cat,val
str,datetime[μs],str,str
"""0xB632D4A2599EAC631263CCEA0E6F…",2016-10-04 07:29:00,"""IMG2669""","""test ordered"""
"""0x7194DEA59D312A1CC699E1195983…",2015-12-12 07:44:00,"""CH 1V""","""test ordered"""
"""0xC1D878516F4EA5823A35900A12F9…",2016-12-01 17:41:00,"""CTHDWO""","""test ordered"""
"""0x74C565DD97AF8C76F890A19184DE…",2016-02-21 06:09:00,"""CTHDWO""","""test ordered"""
"""0x3FAD13B090A102284142436E682A…",2016-03-09 09:21:00,"""MRCPBPWO""","""test ordered"""


In [ ]:
group_3_0 = group_3.filter(pl.col("dt_count") == 0)

group_3_0 = group_3_0.with_columns(
    pl.col("row")
    .str.slice(
        pl.col("row")
        .str.find(datetime_regex)
    )
    .str.splitn(",", 5)
    .struct.rename_fields(["RESULT_TIME", "RESULT_TYPE", "RESULT_FLAG", "ORD_VALUE", "OTHER"])
    .alias("t_break")
).unnest("t_break").drop(["RESULT_TIME", "RESULT_TYPE", "RESULT_FLAG", "OTHER"])

group_3_0.head()

row,CSN_HASH,CPT_CODE,dt_count,ORD_VALUE
str,str,str,u32,str
"""0x338B093CC4FD95901B062BB9B11F…","""0x338B093CC4FD95901B062BB9B11F…","""LAB53611""",0,"""14"""
"""0x4838EC67B2935BA9E0B63D21F05C…","""0x4838EC67B2935BA9E0B63D21F05C…","""LAB13446""",0,"""48"""
"""0xD09279E24C00459C7A5F8B970814…","""0xD09279E24C00459C7A5F8B970814…","""LAB13446""",0,"""44"""
"""0x49D78EA9C975AA47DEAC8ACC0A2C…","""0x49D78EA9C975AA47DEAC8ACC0A2C…","""LAB13446""",0,"""22"""
"""0x0AD9FC9D41B7AB631C37A1336520…","""0x0AD9FC9D41B7AB631C37A1336520…","""LAB2100""",0,"""0.29"""


In [110]:
fields = ["TEST_RESULT_TIME", "TEST_REQUEST_TIME", "TEST_START_TIME"]

group_3_0 = group_3_0.with_columns(
    pl.col("row")
    .str.extract_all(datetime_regex)
    .list.to_struct(fields=fields)
    .alias("dts")            
).unnest("dts")

group_3_0 = group_3_0.with_columns(
    pl.col("^.*_TIME$")
    .str.to_datetime()
)

group_3_0 = group_3_0.select(
    pl.col("CSN_HASH"), 
    pl.min_horizontal(fields).alias("datetime_val"),
    pl.col("CPT_CODE").alias("cat"),
    pl.lit("test ordered").alias("val")
).vstack(
    group_3_0.select(
        pl.col("CSN_HASH"), 
        pl.max_horizontal(fields).alias("datetime_val"),
        pl.col("CPT_CODE").alias("cat"),
        pl.col("ORD_VALUE").alias("val")
    )
).unique()

group_3_0.head()

CSN_HASH,datetime_val,cat,val
str,datetime[μs],str,str
"""0xEB022C5D921F92C2A3A180F7B3A1…",2015-04-09 11:04:00,"""LAB955""","""test ordered"""
"""0x4A50C26B97BE59E0A7F2DF81F96E…",2016-05-04 13:03:00,"""LAB101760""","""1"""
"""0xDCA81555E53DEF32C674E5C72C82…",2015-09-17 00:52:00,"""LAB10120""","""test ordered"""
"""0xF19243C05040EB11F21BBBD4BA26…",2016-04-17 19:55:00,"""LAB101760""","""test ordered"""
"""0x4A50C26B97BE59E0A7F2DF81F96E…",2016-05-03 10:16:00,"""LAB101761""","""test ordered"""


In [111]:
group_3_0.tail()

CSN_HASH,datetime_val,cat,val
str,datetime[μs],str,str
"""0x8D5B762BC51C7B98C6DAFEB0423F…",2016-05-02 10:38:00,"""LAB101760""","""1"""
"""0x29216A98CB98380363EFB3E2CE43…",2016-04-07 03:13:00,"""LAB101760""","""test ordered"""
"""0xBDF229A080940F0744CDF3FBEFCD…",2015-04-17 09:53:00,"""LAB955""","""Stool"""
"""0x8D5B762BC51C7B98C6DAFEB0423F…",2016-05-01 19:24:00,"""LAB101760""","""test ordered"""
"""0x374D9692BFD0E0FAF75EE80EBF9F…",2016-03-08 19:21:00,"""LAB12076""","""test ordered"""


#### 6. Group 1

In [112]:
group_1 = group_1.with_columns(
    pl.col("row").str.count_matches(12 * ',NULL' + ',', literal=True).alias("12_NULL_count")
)

group_1['12_NULL_count'].value_counts().sort("count", descending=True)

12_NULL_count,count
u32,u32
1,3734


Since there are 12 consecutie `NULL` in a series then transform this subset all as "test ordered" only.

In [128]:
group_1 = group_1.with_columns(
    pl.col("row")
    .str.extract(datetime_regex, 0)
    .alias("_TIME")            
)

group_1 = group_1.with_columns(
    pl.col("^.*_TIME$")
    .str.to_datetime()
)

group_1 = group_1.select(
    pl.col("CSN_HASH"), 
    pl.col("_TIME").alias("datetime_val"),
    pl.col("CPT_CODE").alias("cat"),
    pl.lit("test ordered").alias("val")
).unique()

group_1.head()

CSN_HASH,datetime_val,cat,val
str,datetime[μs],str,str
"""0x154A689F405B6BF88CEE2B37A8F1…",2015-10-22 13:47:00,"""GI2102""","""test ordered"""
"""0x3DA0474F522C2D7B97D4F1D53BB7…",2015-05-12 14:42:00,"""GI2100""","""test ordered"""
"""0x3292AF0875932908836E99E24148…",2016-02-25 11:47:00,"""GI2""","""test ordered"""
"""0x795BE7CBBE8006D633A17DCF387D…",2016-12-07 17:09:00,"""PRO21013""","""test ordered"""
"""0xD5EF8AA2E8F33C0E3CB7A8B2A4B4…",2016-01-19 17:17:00,"""GI210100""","""test ordered"""


#### 7. Reconstitute 

In [131]:
df_out = pl.concat(
    [
        group_1, 
        group_2, 
        group_3_0, 
        group_3_1, 
        group_4
    ]
).unique()

df_out.shape

(23210213, 4)

In [132]:
23213049 - 23210213

2836

In [130]:
df_out.head()

CSN_HASH,datetime_val,cat,val
str,datetime[μs],str,str
"""0x154A689F405B6BF88CEE2B37A8F1…",2015-10-22 13:47:00,"""GI2102""","""test ordered"""
"""0x3DA0474F522C2D7B97D4F1D53BB7…",2015-05-12 14:42:00,"""GI2100""","""test ordered"""
"""0x3292AF0875932908836E99E24148…",2016-02-25 11:47:00,"""GI2""","""test ordered"""
"""0x795BE7CBBE8006D633A17DCF387D…",2016-12-07 17:09:00,"""PRO21013""","""test ordered"""
"""0xD5EF8AA2E8F33C0E3CB7A8B2A4B4…",2016-01-19 17:17:00,"""GI210100""","""test ordered"""


In [116]:
df_out.tail()

CSN_HASH,datetime_val,cat,val
str,datetime[μs],str,str
"""0x5A8276FB63D77813AE4FB025C547…",2014-07-06 14:44:00,"""LAB144""","""test ordered"""
"""0x2C3E077F355F3A99F58BD94A3D3D…",2015-04-10 16:00:00,"""LAB15""","""test ordered"""
"""0x2C3CA8E8E3CA151990B3CD43BD99…",2016-04-23 15:12:00,"""LAB56""","""test ordered"""
"""0x1242DDF9A9DC1C1978B6E99F7EF8…",2016-09-20 16:28:00,"""LAB347""","""test ordered"""
"""0x4CF38D8B907C96C44AFB3234483F…",2015-06-04 07:30:00,"""LAB47""","""61"""
